Load libraries

In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Ochtend

Load en bekijk data

In [48]:
data = pd.read_csv('titanic_data.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Bekijk de datatypes

In [49]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Wat is de minimum, maximum en gemiddelde leeftijd van de passagiers?

In [50]:
import math 
print(min(data.get('Age')))
print(max(data.get('Age')))

df_without_NaN = data.dropna(subset=['Age'])

print(sum(df_without_NaN.get('Age') / len(df_without_NaN.get('Age'))))



0.42
80.0
29.69911764705899


Wat is het percentage van mensen die de ramp niet hebben overleefd?

In [51]:
survived = sum(data.get('Survived'))
total = len(data.get('PassengerId'))
print((survived / total) * 100)


38.38383838383838


Wat is de minimum, gemiddelde, en maximum prijs (Fare) in de verschillende klassen (Pclass)?

In [52]:
print(data.groupby('Pclass')['Fare'].agg(['min','mean','max']))

        min       mean       max
Pclass                          
1       0.0  84.154687  512.3292
2       0.0  20.662183   73.5000
3       0.0  13.675550   69.5500


Welke data is categorisch?

In [53]:
data.select_dtypes(include=[object]).columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

Verijder de kolommen 'Name', 'Cabin' en 'Ticket'

In [54]:
data.drop(['Name','Cabin','Ticket'], axis=1)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
888,889,0,3,female,NaN,1,2,23.4500,S
889,890,1,1,male,26.0,0,0,30.0000,C


Codeer categorische features met indices (getallen)

In [55]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Select categorical columns
categorical_columns = data.select_dtypes(include=[object]).columns

# Encode categorical features
for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column])

# Display encoded DataFrame
print(data)

     PassengerId  Survived  Pclass  Name  Sex   Age  SibSp  Parch  Ticket  \
0              1         0       3   108    1  22.0      1      0     523   
1              2         1       1   190    0  38.0      1      0     596   
2              3         1       3   353    0  26.0      0      0     669   
3              4         1       1   272    0  35.0      1      0      49   
4              5         0       3    15    1  35.0      0      0     472   
..           ...       ...     ...   ...  ...   ...    ...    ...     ...   
886          887         0       2   548    1  27.0      0      0     101   
887          888         1       1   303    0  19.0      0      0      14   
888          889         0       3   413    0   NaN      1      2     675   
889          890         1       1    81    1  26.0      0      0       8   
890          891         0       3   220    1  32.0      0      0     466   

        Fare  Cabin  Embarked  
0     7.2500    147         2  
1    71.283

Vervang NaN in "Ages". Bepaal zelf of mediaan of gemiddelde hiervoor het best is.

In [56]:
#Ik heb voor het gemiddelde gekozen, omdat het een rond getal.
mean_age = data['Age'].mean()

data.fillna({'Age': mean_age}, inplace=True)

data.get('Age')

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

## Middag

Voorspel of passagiers de ramp hebben overleefd of niet. Bereid de data voor om je model te trainen. Wat wordt je target vector en hoe ziet je feature matrix er uit?

In [57]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Select target vector
target = data['Survived']

# Select features and create a copy
features = data[['Age', 'Sex', 'Pclass']].copy()

# Encode categorical feature 'Sex' with LabelEncoder
label_encoder = LabelEncoder()
features['Sex'] = label_encoder.fit_transform(features['Sex'])

# Handle missing values in 'Age' by imputing with mean
imputer = SimpleImputer(strategy='mean')
features['Age'] = imputer.fit_transform(features[['Age']])

# Display prepared data
print("Target vector:")
print(target)

print("\nFeature matrix:")
print(features)


Target vector:
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

Feature matrix:
           Age  Sex  Pclass
0    22.000000    1       3
1    38.000000    0       1
2    26.000000    0       3
3    35.000000    0       1
4    35.000000    1       3
..         ...  ...     ...
886  27.000000    1       2
887  19.000000    0       1
888  29.699118    0       3
889  26.000000    1       1
890  32.000000    1       3

[891 rows x 3 columns]


Train en test KNN classifier

In [58]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize the features (recommended for KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Instantiate the KNN classifier with a chosen number of neighbors (e.g., k=5)
knn_classifier = KNeighborsClassifier(n_neighbors=5)

# Train the classifier
knn_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test_scaled)

# Calculate accuracy2
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.776536312849162


    Train en test regressie classifier. Kies je lineair of logistisch?

In [59]:
from sklearn.linear_model import LogisticRegression

#Instantiate the logistic regression classifier
logistic_classifier = LogisticRegression()

#Train the classifier
logistic_classifier.fit(X_train_scaled, y_train)

#Make predictions on the test set
y_pred_logistic = logistic_classifier.predict(X_test_scaled)

#Calculate accuracy for logistic regression classifier
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print("Accuracy (Logistic Regression):", accuracy_logistic)

Accuracy (Logistic Regression): 0.8100558659217877


Welk model geeft het beste resultaat?

Regressief logistiek

Extra: Kan je het resultaat verbeteren?

Ja door de features aan te passen.